### Authentication API

In [130]:
import requests
from requests.auth import HTTPBasicAuth
import json

url = "https://datahub.eu-latest.cumulocity.com/apiv2/login"

payload = {
  "userName": "t15058219/demo",
  "password": "Manage2020"
}

headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data = json.dumps(payload))

In [131]:
token_id = json.loads(response.text)['token']

In [132]:
token_id

'df3rlacn8j4ddoadc3vpq0b6gs'

### Show resources

In [133]:
tenantID = payload['userName'].split("/")[0]
params = "?showDatasets=true&showSources=true&showSpaces=true&showHomes=true"

url = "https://datahub.eu-latest.cumulocity.com/apiv2/resourcetree/"+tenantID+"DataLake"+params

headers = {
  'Authorization': '_dremio'+token_id
}

response = requests.request("GET", url, headers=headers)
if 'url' in json.loads(response.text)['resources'][0].keys():
    while 'url' in json.loads(response.text)['resources'][0].keys():
        tempURL = json.loads(response.text)['resources'][0]['url']
        url = "https://datahub.eu-latest.cumulocity.com/apiv2"+tempURL+params
        response = requests.request("GET", url, headers=headers)

resources = json.loads(response.text)
resources

{'resources': [{'type': 'PHYSICAL_DATASET_SOURCE_FOLDER',
   'name': 'alarms',
   'fullPath': ['t15058219DataLake',
    'sag-customer-poc',
    't15058219',
    'alarms']},
  {'type': 'FOLDER',
   'name': 'c8y_cdh_temp',
   'fullPath': ['t15058219DataLake',
    'sag-customer-poc',
    't15058219',
    'c8y_cdh_temp'],
   'url': '/resourcetree/t15058219DataLake.%22sag-customer-poc%22.t15058219.c8y_cdh_temp'},
  {'type': 'PHYSICAL_DATASET_SOURCE_FOLDER',
   'name': 'events',
   'fullPath': ['t15058219DataLake',
    'sag-customer-poc',
    't15058219',
    'events']},
  {'type': 'PHYSICAL_DATASET_SOURCE_FOLDER',
   'name': 'measurements_c8y_Acceleration',
   'fullPath': ['t15058219DataLake',
    'sag-customer-poc',
    't15058219',
    'measurements_c8y_Acceleration']}]}

### SQL

In [134]:
url = "https://datahub.eu-latest.cumulocity.com/api/v3/sql"

sqlQuery = {"sql": 'Select * from t15058219DataLake."sag-customer-poc".t15058219.events'}

payload = json.dumps(sqlQuery)

headers = {
  'Authorization': '_dremio'+token_id,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data = payload)

In [135]:
sqlID = json.loads(response.text)['id']

In [136]:
sqlID

'20fbb5cc-72cb-8867-9e58-8a578df76d00'

### Get Job Status

In [27]:
url = "https://datahub.eu-latest.cumulocity.com/api/v3/job/"+sqlID

headers = {
  'Authorization': '_dremio'+token_id
}

response = requests.request("GET", url, headers=headers)

In [30]:
jobState = json.loads(response.text)['jobState']

In [38]:
jobState

'COMPLETED'

### Get results (data)

In [31]:
url = "https://datahub.eu-latest.cumulocity.com/api/v3/job/"+sqlID+"/results"

headers = {
  'Authorization': '_dremio'+token_id
}

if jobState == 'COMPLETED':
    response = requests.request("GET", url, headers=headers)

In [35]:
import pandas as pd
data = pd.DataFrame(json.loads(response.text)['rows'])

In [36]:
data.head()

,id,creationTime,creationTimeOffset,creationTimeWithOffset,lastUpdated,lastUpdatedOffset,lastUpdatedWithOffset,YEAR,MONTH,DAY,time,timeOffset,timeWithOffset,source,text,type,dir0,dir1,dir2,dir3
0,4039841,2020-06-01 13:19:21.553,0,2020-06-01 13:19:21.553,2020-06-01 13:19:21.553,0,2020-06-01 13:19:21.553,2020,06,01,2020-06-01 13:19:21.553,0,2020-06-01 13:19:21.553,922629,This is a warning,Testing,monthly_2020_06,2020,06,None
1,4039872,2020-06-01 13:21:51.559,0,2020-06-01 13:21:51.559,2020-06-01 13:21:51.559,0,2020-06-01 13:21:51.559,2020,06,01,2020-06-01 13:21:51.559,0,2020-06-01 13:21:51.559,922629,This is a warning,Testing,monthly_2020_06,2020,06,None
2,4039903,2020-06-01 13:24:21.554,0,2020-06-01 13:24:21.554,2020-06-01 13:24:21.554,0,2020-06-01 13:24:21.554,2020,06,01,2020-06-01 13:24:21.554,0,2020-06-01 13:24:21.554,922629,This is a warning,Testing,monthly_2020_06,2020,06,None
3,4039934,2020-06-01 13:26:51.579,0,2020-06-01 13:26:51.579,2020-06-01 13:26:51.579,0,2020-06-01 13:26:51.579,2020,06,01,2020-06-01 13:26:51.579,0,2020-06-01 13:26:51.579,922629,This is a warning,Testing,monthly_2020_06,2020,06,None
4,4039965,2020-06-01 13:29:21.571,0,2020-06-01 13:29:21.571,2020-06-01 13:29:21.571,0,2020-06-01 13:29:21.571,2020,06,01,2020-06-01 13:29:21.571,0,2020-06-01 13:29:21.571,922629,This is a warning,Testing,monthly_2020_06,2020,06,None
